In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score 
# On importe la métrique "silhouette" qui nous servira à déterminer le nombre de clusters optimal.

%config InlineBackend.figure_format = 'png'

In [2]:
df = pd.read_csv('data/df_train.csv')

In [3]:
len(df)

1855314

In [4]:
# Recodage de certaines modalités afin de réordonner par ordre croissant d'engagement
df["contextid"] = df["contextid"].replace(0,1)
df["contextid"] = df["contextid"].replace(10,3)

# Remplacement des valeurs manquantes par des 0 dans les variables commençant par ltf
df['ltf_lastpartnerclicktimestamp'] = df['ltf_lastpartnerclicktimestamp'].fillna(0)
df['ltf_nbglobalclick_4w'] = df['ltf_nbglobalclick_4w'].fillna(0)
df['ltf_nbglobaldisplay_4w'] = df['ltf_nbglobaldisplay_4w'].fillna(0)
df['ltf_nbglobaldisplaysincelastpartnerproductview'] = df['ltf_nbglobaldisplaysincelastpartnerproductview'].fillna(0)
df['ltf_nbpartnerdisplayssincelastclick'] = df['ltf_nbpartnerdisplayssincelastclick'].fillna(0)
df['ltf_nbpartnerclick_4w'] = df['ltf_nbpartnerclick_4w'].fillna(0)
df['ltf_nbpartnerdisplay_4w'] = df['ltf_nbpartnerdisplay_4w'].fillna(0)
df['ltf_nbpartnersales_4w'] = df['ltf_nbpartnersales_4w'].fillna(0)
df['ltf_nbpartnerdisplay_90d'] = df['ltf_nbpartnerdisplay_90d'].fillna(0)
df['ltf_nbpartnerclick_90d'] = df['ltf_nbpartnerclick_90d'].fillna(0)
df['ltf_nbpartnersales_90d'] = df['ltf_nbpartnersales_90d'].fillna(0)
# Idem ici
df['nbdisplay_1hour'] = df['nbdisplay_1hour'].fillna(0)
df['nbdisplaypartnerapprox_1d_sum_xdevice'] = df['nbdisplaypartnerapprox_1d_sum_xdevice'].fillna(0)
df['nbdisplayaffiliateapprox_1d_sum_xdevice'] = df['nbdisplayaffiliateapprox_1d_sum_xdevice'].fillna(0)
df['nbdisplayglobalapprox_1d_sum_xdevice'] = df['nbdisplayglobalapprox_1d_sum_xdevice'].fillna(0)	
df['campaignctrlast24h'] = df['campaignctrlast24h'].fillna(0)

In [5]:
# Suppression de l'appareil cliquant un nombre aberrant de fois
df.drop(df.index[df['hashed_xd_id'] == r"b'\x12\xb97|\xbe~\\\x94\xe8\xa7\r\x9d#\x92\x95#\xd1J\xfa\x95G\x93\x13\x0f\x8a9Y\xc7\xb8I\xac\xa8'"], inplace = True)

In [6]:
df['display_size'] = df['display_width']*df['display_height']

In [7]:
# On enlève des variables ltf grâce aux corrélations visibles dans la matrice. 
df_clusters = df[['contextid','dayssincelastvisitdouble','campaignctrlast24h','nbdisplay_1hour','nbdayssincelastclick','display_size','ltf_nbpartnerdisplayssincelastclick','ltf_nbglobaldisplay_4w','ltf_nbpartnerclick_90d','ltf_nbpartnerdisplay_90d','ltf_nbpartnersales_90d','nbdisplayglobalapprox_1d_sum_xdevice']]

In [8]:
# On va chercher à obtenir des groupes d'observations de caractéristiques proches afin de voir si l'on peut distinguer les clics des non-clics. 
sc = StandardScaler()
Z = sc.fit_transform(df_clusters)
Z

array([[ 0.14873565, -0.40809959, -0.8906595 , ..., -0.07290974,
        -0.01397608, -0.32103262],
       [-1.92104218, -0.44867976, -0.360678  , ..., -0.07290974,
        -0.01397608,  0.8070464 ],
       [ 0.14873565, -0.33748309,  1.82717896, ..., -0.07290974,
        -0.01397608, -0.5009203 ],
       ...,
       [ 0.97664678, -0.36785013, -0.92834265, ..., -0.07290974,
        -0.01397608, -0.51352702],
       [ 1.39060235, -0.4168148 ,  2.20278178, ..., -0.07290974,
        -0.01397608, -0.34134216],
       [ 0.14873565, -0.1664472 , -0.50158415, ..., -0.07290974,
        -0.01397608, -0.45594838]])

In [9]:
# Vérification que les moyennes sont nulles et les écarts-types unitaires. 
print(np.mean(Z, axis=0))
print(np.std(Z, axis=0, ddof=0))

[-4.69520271e-17  2.45057639e-16 -5.90496082e-17 -2.05773590e-17
 -9.95337090e-18  1.23711223e-17  1.66154232e-17 -1.56712648e-17
  9.90263342e-18  4.87962244e-18 -2.46683444e-18 -3.14861388e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [10]:
# On applique la méthode des k-means sur les variables centrées et réduites, en supposant pour l'instant par défaut qu'il existe 2 clusters.
kmeans = KMeans(n_clusters=2, random_state=146355806) 
# random_state est la graine aléatoire specifiée pour rendre les résultats reproductibles.
kmeans.fit(Z)

# On trie les index en fonction des différents groupes de députés (ie des clusters).
idk = np.argsort(kmeans.labels_)

# Affichage des observations et de leurs groupes.
print(pd.DataFrame({"Observation" : df_clusters.index[idk], "Cluster" : kmeans.labels_[idk]}))

         Observation  Cluster
0                  0        0
1            1236366        0
2            1236364        0
3            1236363        0
4            1236362        0
5            1236360        0
6            1236359        0
7            1236358        0
8            1236357        0
9            1236356        0
10           1236368        0
11           1236355        0
12           1236352        0
13           1236350        0
14           1236349        0
15           1236347        0
16           1236346        0
17           1236345        0
18           1236344        0
19           1236343        0
20           1236342        0
21           1236354        0
22           1236369        0
23           1236370        0
24           1236371        0
25           1236392        0
26           1236391        0
27           1236390        0
28           1236389        0
29           1236388        0
...              ...      ...
1610464      1122020        1
1610465   

In [11]:
nb_clusters = kmeans.labels_.tolist()
print(nb_clusters.count(0)) # Comptage du nombre de députés dans le cluster 1.
print(nb_clusters.count(1)) # Comptage du nombre de députés dans le cluster 2.

1604453
6041


Le découpage en deux clusters ne nous permettra pas de discriminer les clics des non-clics ; le deuxième cluster est en effet trop petit. 
Essayons avec 3 clusters. 

In [12]:
# On va faire varier le nombre de clusters dans un intervalle réaliste, disons de 2 à 10.
# res = np.arange(9, dtype="double")
# for k in np.arange(9):
    # km = KMeans(n_clusters=k+2)
    # km.fit(Z)
    # res[k] = silhouette_score(Z, km.labels_)
# print(res)

In [13]:
# plt.figure(figsize=(8,4))
# plt.plot(np.arange(2,11,1), res)
# plt.title("Coefficient de silhouette moyen en fonction du nombre de clusters", size=16)
# plt.xlabel("Nombre de clusters", size=10)
# plt.ylabel("Score de silhouette moyen", size=10)
# plt.show()

In [14]:
kmeans = KMeans(n_clusters=3, random_state=146355806) 
kmeans.fit(Z)
idk = np.argsort(kmeans.labels_)
print(pd.DataFrame({"Observation" : df_clusters.index[idk], "Cluster" : kmeans.labels_[idk]}))

         Observation  Cluster
0            1181351        0
1             238204        0
2             238205        0
3             238208        0
4            1463589        0
5            1656917        0
6            1463590        0
7            1146803        0
8            1285265        0
9             835818        0
10           1656908        0
11            464477        0
12           1656903        0
13            238225        0
14           1285262        0
15            464486        0
16            464471        0
17            835808        0
18            672321        0
19            672322        0
20           1463603        0
21           1146819        0
22           1285259        0
23            238239        0
24           1656890        0
25           1463607        0
26            672326        0
27           1146825        0
28            238249        0
29           1656881        0
...              ...      ...
1610464       669199        2
1610465   

In [15]:
nb_clusters1 = kmeans.labels_.tolist()
print(nb_clusters1.count(0)) 
print(nb_clusters1.count(1))
print(nb_clusters1.count(2))

239141
6029
1365324


Etudions les caractéristiques des observations dans chacun des clusters. 

In [16]:
A = pd.DataFrame({"Observation" : df_clusters.index[idk], "Cluster" : kmeans.labels_[idk]})
L1 = [] # Liste qui contiendra les index des députés du cluster 1.
L2 = [] # Liste qui contiendra les index des députés du cluster 2.
L3 = [] # Liste qui contiendra les index des députés du cluster 3.
for i in range(len(A)) :
    if A['Cluster'][i] == 0 :
        L1.append(A['Observation'][i])
    if A['Cluster'][i] == 1 :
        L2.append(A['Observation'][i])
    if A['Cluster'][i] == 2 : 
        L3.append(A['Observation'][i])

In [17]:
df_cluster_1 = df.loc[L1] # Sous-table avec les députés du cluster 1.
df_cluster_2 = df.loc[L2] # Sous-table avec les députés du cluster 2.
df_cluster_3 = df.loc[L3] # Sous-table avec les députés du cluster 3.

In [18]:
len(L1), len(L2), len(L3)

(239141, 6029, 1365324)

On retrouve bien le nombre d'observations présentes dans chaque cluster. 

In [19]:
display(pd.DataFrame({'Nb dans cluster 1' : df_cluster_1['is_display_clicked'].value_counts(), 'Prop dans cluster 1' : df_cluster_1['is_display_clicked'].value_counts(normalize=True)})) 
# Répartition des clics et des non clics dans le cluster 1..
display(pd.DataFrame({'Nb dans cluster 2' : df_cluster_2['is_display_clicked'].value_counts(), 'Prop dans cluster 2' : df_cluster_2['is_display_clicked'].value_counts(normalize=True)})) 
# Répartition des clics et des non clics dans le cluster 2.
display(pd.DataFrame({'Nb dans cluster 3' : df_cluster_3['is_display_clicked'].value_counts(), 'Prop dans cluster 3' : df_cluster_3['is_display_clicked'].value_counts(normalize=True)}))
# Répartition des clics et des non clics dans le cluster 3.

,Nb dans cluster 1,Prop dans cluster 1
0,230664,0.964552
1,8477,0.035448


,Nb dans cluster 2,Prop dans cluster 2
0,5583,0.926024
1,446,0.073976


,Nb dans cluster 3,Prop dans cluster 3
0,1282495,0.939334
1,82829,0.060666


In [20]:
df_cluster_1.describe()

,Unnamed: 0,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,googleviewability,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,2.391410e+05,239141.000000,239141.000000,237865.000000,237865.000000,237865.000000,2.075390e+05,2.075390e+05,2.075390e+05,64935.000000,...,239141.000000,239141.000000,239141.000000,239141.000000,239141.000000,239141.000000,2.391410e+05,239141.000000,239141.000000,2.391410e+05
mean,9.318692e+05,6.037660,1.073275,37.068900,0.729355,94.218649,1.105811e+08,-1.762402e+08,-1.874093e+08,59.410919,...,35.259333,58.694222,126.633865,0.311313,362.944589,250.710196,1.602853e+09,0.035448,0.935272,9.196404e+04
std,5.377531e+05,2.455203,0.260588,33.767617,0.791453,1037.636807,1.217655e+09,1.124695e+09,1.153933e+09,34.503522,...,20.218316,87.052084,122.536115,0.189056,277.651188,234.852831,1.775784e+05,0.184909,2.538710,9.456407e+04
min,3.000000e+00,1.000000,1.000000,3.000000,0.000000,-1.000000,-2.145916e+09,-2.147307e+09,-2.147255e+09,-1.000000,...,0.000000,0.000000,13.000000,0.034118,0.000000,0.000000,1.602547e+09,0.000000,0.000000,0.000000e+00
25%,4.662540e+05,6.000000,1.000000,4.000000,0.000000,12.000000,-8.462803e+08,-1.089605e+09,-1.089605e+09,30.000000,...,21.000000,10.826296,47.265278,0.136416,300.000000,50.000000,1.602694e+09,0.000000,0.050000,1.600000e+04
50%,9.362870e+05,6.000000,1.000000,29.000000,1.000000,12.000000,3.640997e+08,-2.180814e+08,-5.237546e+08,68.000000,...,34.000000,28.000000,83.751250,0.254042,300.000000,250.000000,1.602849e+09,0.000000,0.254518,7.500000e+04
75%,1.397287e+06,8.000000,1.000000,73.000000,1.000000,127.000000,1.340720e+09,6.404254e+08,6.397927e+08,90.000000,...,48.868148,67.000000,169.420000,0.512843,360.000000,480.000000,1.603022e+09,0.000000,0.822104,1.800000e+05
max,1.856864e+06,9.000000,2.000000,134.000000,2.000000,31770.000000,2.146753e+09,2.147353e+09,2.147359e+09,100.000000,...,460.277870,2269.557407,3953.888889,0.790542,1280.000000,1286.000000,1.603152e+09,1.000000,100.000000,1.028800e+06


In [21]:
df_cluster_2.describe()

,Unnamed: 0,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,googleviewability,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,6.029000e+03,6029.000000,6029.000000,6012.000000,6012.000000,6012.000000,5.379000e+03,5.379000e+03,5.379000e+03,1860.000000,...,6029.000000,6029.000000,6029.000000,6029.000000,6029.000000,6029.000000,6.029000e+03,6029.000000,6029.000000,6.029000e+03
mean,9.217121e+05,7.374026,1.137502,34.008816,0.738689,69.295576,1.370534e+08,-9.072127e+07,-1.385818e+08,60.796237,...,25.175368,30.955367,82.695714,0.306986,327.919721,241.687842,1.602856e+09,0.073976,1.567993,8.842949e+04
std,5.380897e+05,1.841765,0.344405,32.836823,0.794307,562.086580,1.259048e+09,1.138867e+09,1.155705e+09,30.823743,...,21.053366,65.393863,99.767500,0.189240,266.864481,231.251260,1.767350e+05,0.261753,3.776451,9.744088e+04
min,9.400000e+01,1.000000,1.000000,3.000000,0.000000,1.000000,-2.140960e+09,-2.146561e+09,-2.146412e+09,-1.000000,...,0.000000,0.000000,0.000000,0.034118,0.000000,0.000000,1.602550e+09,0.000000,0.008475,0.000000e+00
25%,4.581640e+05,6.000000,1.000000,4.000000,0.000000,12.000000,-8.998495e+08,-1.005008e+09,-1.089605e+09,41.000000,...,6.901435,2.000000,28.000000,0.139050,160.000000,50.000000,1.602698e+09,0.000000,0.118000,1.600000e+04
50%,9.188420e+05,8.000000,1.000000,16.000000,1.000000,16.000000,3.640997e+08,-6.162982e+07,-2.627962e+08,68.000000,...,21.395000,10.146481,54.358981,0.245336,300.000000,250.000000,1.602850e+09,0.000000,0.471000,7.500000e+04
75%,1.389878e+06,9.000000,1.000000,73.000000,1.000000,127.000000,1.384186e+09,7.282313e+08,6.563225e+08,86.000000,...,41.000000,32.000000,98.468750,0.512220,336.000000,300.000000,1.603023e+09,0.000000,1.420000,1.536000e+05
max,1.856705e+06,9.000000,2.000000,134.000000,2.000000,30724.000000,2.141561e+09,2.146829e+09,2.144924e+09,99.000000,...,193.195278,1341.353333,1416.062778,0.790542,1280.000000,1280.000000,1.603149e+09,1.000000,84.459663,1.024000e+06


In [22]:
df_cluster_3.describe()

,Unnamed: 0,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,googleviewability,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,1.365324e+06,1.365324e+06,1.365324e+06,1.359706e+06,1.359706e+06,1.359706e+06,1.246628e+06,1.246628e+06,1.246628e+06,434862.00000,...,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06,1.365324e+06
mean,9.287308e+05,5.563513e+00,1.110386e+00,3.655437e+01,7.276036e-01,9.472602e+01,9.744425e+07,-7.275786e+07,-1.089303e+08,59.22234,...,1.071949e+01,1.062098e+01,2.984569e+01,2.532534e-01,2.857652e+02,2.241796e+02,1.602857e+09,6.066619e-02,1.140309e+00,8.085265e+04
std,5.352432e+05,2.401246e+00,3.133698e-01,3.339118e+01,8.085108e-01,9.837410e+02,1.260949e+09,1.157488e+09,1.173646e+09,30.52539,...,1.384880e+01,2.035892e+01,3.424088e+01,1.837486e-01,2.498844e+02,2.276490e+02,1.745974e+05,2.387171e-01,3.030289e+00,9.753806e+04
min,0.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,-1.000000e+00,-2.147479e+09,-2.147454e+09,-2.147479e+09,-1.00000,...,0.000000e+00,0.000000e+00,0.000000e+00,3.411759e-02,0.000000e+00,0.000000e+00,1.602547e+09,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.658448e+05,5.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,1.200000e+01,-9.718820e+08,-9.935616e+08,-1.017692e+09,37.00000,...,1.000000e+00,0.000000e+00,6.000000e+00,1.029036e-01,0.000000e+00,0.000000e+00,1.602700e+09,0.000000e+00,7.900000e-02,0.000000e+00
50%,9.277815e+05,6.000000e+00,1.000000e+00,2.900000e+01,0.000000e+00,1.600000e+01,2.603107e+08,-6.162982e+07,-2.856484e+08,66.00000,...,5.000000e+00,3.000000e+00,1.751211e+01,1.838390e-01,3.000000e+02,2.500000e+02,1.602855e+09,0.000000e+00,3.180814e-01,7.500000e+04
75%,1.392135e+06,7.000000e+00,1.000000e+00,7.300000e+01,1.000000e+00,1.270000e+02,1.384186e+09,8.349926e+08,7.830828e+08,85.00000,...,1.469028e+01,1.100000e+01,4.100000e+01,4.015411e-01,3.200000e+02,2.800000e+02,1.603021e+09,0.000000e+00,9.850001e-01,1.000000e+05
max,1.856872e+06,9.000000e+00,2.000000e+00,1.340000e+02,2.000000e+00,3.177000e+04,2.147366e+09,2.147451e+09,2.147451e+09,100.00000,...,1.890000e+02,3.036156e+02,3.052078e+02,7.905422e-01,1.374000e+03,1.286000e+03,1.603155e+09,1.000000e+00,1.000000e+02,1.101600e+06
